#### Provide a frequency distribution of the vital signs/physical exam readings observed across the dataset

In [68]:
import os
import glob
import xml.etree.ElementTree as ET
import re
import pandas as pd
from collections import Counter

In [69]:
## Finding all patterns of vital signs in the text

directory = 'Project3_data'
xml_files = glob.glob(os.path.join(directory, '*.xml'))
vital_signs_pattern = {}
count = 0
files_with_vital_signs = []
for xml_file in xml_files:
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for elem in root.iter('TEXT'):
        if elem.text:
            matches = re.findall(r'\b(vital signs|Vital Signs|VITAL SIGNS|Vital signs|VITAL signs)\b', elem.text)
            if matches:
                files_with_vital_signs.append(xml_file)
            # vital_signs_pattern.update(matches)
            if matches:
                for match in matches:
                    if match in vital_signs_pattern:
                        vital_signs_pattern[match].append(xml_file)
                    else:
                        vital_signs_pattern[match] = [xml_file]

print(vital_signs_pattern.keys())
## Counter({'Vital Signs': 139, 'Vital signs': 61, 'VITAL SIGNS': 42, 'vital signs': 36})

dict_keys(['VITAL SIGNS', 'vital signs', 'Vital Signs', 'Vital signs'])


In [73]:
# Define the patterns for the vital signs
vital_signs_patterns = {
    'temperature': [
        r'[Tt]emperature (\d+\.?\d*)',
        r'[Tt]emperature is about (\d+\.?\d*)',
        r'[Tt]emperature is (\d+\.?\d*)',
        r'[Tt]emperature of (\d+\.?\d*)',
        r'TEMPERATURE (\d+\.?\d*)',
    ],
    'pulse': [
        r'Pulse (\d+)',
        r'PULSE (\d+)',
    ],
    'respiratory rate': [
        r'respiratory rate (\d+)',
        r'RP (\d+)',
    ],
    'blood pressure': [
        r'[Bb]blood pressure (\d+/\d+)',
        r'BLOOD PRESSURE (\d+/\d+)',
        r'BP (\d+/\d+)',
        r'BP: (\d+/\d+)'
    ],
    'oxygen saturation': [
        r'Oxygen saturation (\d+%)',
        r'O2 SAT (\d+)',
    ],
    'stable/normal': [
        r'[Ss]table',
        r'[Nn]ormal',
    ],
    'weight': [
        r'[Ww]eight (\d+%)',
        r'WEIGHT (\d+%)',
        r'[Ww]eight is (\d+%)',
    ]
}
vitalSign_df = pd.DataFrame()
frequency = {sign: 0 for sign in vital_signs_patterns.keys()}
no_matches_found = []
for key, value in vital_signs_pattern.items():
    count = 0
    for file in value:
        match = re.search(r'(\d+)-', file)
        if match:
            patient_number = match.group(1)
        else:
            print(f'Could not extract patient number from file name: {file}')
            continue

        tree = ET.parse(file)
        root = tree.getroot()
        vital_signs = []
        match_found = False
        for elem in root.iter('TEXT'):
            if elem.text:
                for sign, patterns in vital_signs_patterns.items():
                    for pattern in patterns:
                        matches = re.findall(pattern, elem.text, re.IGNORECASE)
                        if matches:
                            frequency[sign] += 1
                            if sign == 'stable/normal':
                                vital_signs.append({'patient_number': patient_number,'sign': sign, 'value': 0})
                            else:
                                vital_signs.append({'patient_number': patient_number,'sign': sign, 'value': matches[0]})
                            match_found = True
        if vital_signs:
            print(type(vitalSign_df))
            print(type(vital_signs))
            vitalSign_df = pd.concat([vitalSign_df, pd.DataFrame(vital_signs)], ignore_index=True)
        if not match_found:
            no_matches_found.append(file)

<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class '

In [74]:
vitalSign_df.head(10)

,patient_number,sign,value
0,379,temperature,97
1,379,temperature,97
2,379,pulse,62
3,379,pulse,62
4,379,respiratory rate,16
5,379,blood pressure,163/96
6,379,oxygen saturation,100%
7,379,stable/normal,0
8,316,stable/normal,0
9,222,stable/normal,0


In [75]:
print(len(vitalSign_df))
print(len(no_matches_found))
print(no_matches_found)
## manually add some vital
new_vital_signs = [
  {'patient_number': '192','sign': 'blood pressure', 'value': '152/84'},
  {'patient_number': '192','sign': 'pulse', 'value': '68'},
  {'patient_number': '192','sign': 'weight', 'value': '263'},
  {'patient_number': '148','sign': 'blood pressure', 'value': '103/80'},
  {'patient_number': '148','sign': 'pulse', 'value': '70'},
  {'patient_number': '148','sign': 'weight', 'value': '164'},
  {'patient_number': '148','sign': 'temperature', 'value': '96.6'},
  {'patient_number': '148','sign': 'respiratory rate', 'value': '16'},
  {'patient_number': '346','sign': 'temperature', 'value': '98.6'},
  {'patient_number': '346','sign': 'pulse', 'value': '99'},
  {'patient_number': '346','sign': 'blood pressure', 'value': '174/65'},
  {'patient_number': '346','sign': 'respiratory rate', 'value': '20'},
  {'patient_number': '346','sign': 'oxygen saturation', 'value': '91%'},
  {'patient_number': '189','sign': 'blood pressure', 'value': '120/68'},
  {'patient_number': '189','sign': 'pulse', 'value': '80'},
  {'patient_number': '236','sign': 'blood pressure', 'value': '128/68'},
  {'patient_number': '236','sign': 'weight', 'value': '107'},
  {'patient_number': '338','sign': 'blood pressure', 'value': '110/61'},
  {'patient_number': '338','sign': 'pulse', 'value': '61'},
  {'patient_number': '338','sign': 'weight', 'value': '220'},
  {'patient_number': '139','sign': 'blood pressure', 'value': '140/70'},
  {'patient_number': '139','sign': 'pulse', 'value': '64'},
  {'patient_number': '139','sign': 'weight', 'value': '151'},
  {'patient_number': '385','sign': 'weight', 'value': '152'},
  {'patient_number': '385','sign': 'blood pressure', 'value': '115/55'},
  {'patient_number': '385','sign': 'pulse', 'value': '70'},
  {'patient_number': '385','sign': 'temperature', 'value': '96.3'},
  {'patient_number': '376','sign': 'weight', 'value': '183'},
  {'patient_number': '376','sign': 'blood pressure', 'value': '130/70'},
  {'patient_number': '397','sign': 'temperature', 'value': '98.3'},
  {'patient_number': '397','sign': 'blood pressure', 'value': '114/73'},
  {'patient_number': '397','sign': 'pulse', 'value': '100'},
  {'patient_number': '397','sign': 'respiratory rate', 'value': '16'},
  {'patient_number': '397','sign': 'oxygen saturation', 'value': '96%'},
  {'patient_number': '278','sign': 'blood pressure', 'value': '130/72'},
  {'patient_number': '278','sign': 'pulse', 'value': '68'},
  {'patient_number': '278','sign': 'weight', 'value': '198'},
  {'patient_number': '242','sign': 'blood pressure', 'value': '142/84'},
  {'patient_number': '242','sign': 'weight', 'value': '178'},
  {'patient_number': '242','sign': 'weight', 'value': '178'},
  {'patient_number': '110','sign': 'pulse', 'value': '66'},
  {'patient_number': '110','sign': 'temperature', 'value': '98.7'},
  {'patient_number': '110','sign': 'oxygen saturation', 'value': '97%'},
  {'patient_number': '110','sign': 'blood pressure', 'value': '132/60'},
  {'patient_number': '244','sign': 'blood pressure', 'value': '120/64'},
  {'patient_number': '244','sign': 'pulse', 'value': '68'},
  {'patient_number': '244','sign': 'respiratory rate', 'value': '14'},
  {'patient_number': '244','sign': 'weight', 'value': '202'},
  {'patient_number': '354','sign': 'blood pressure', 'value': '154/71'},
  {'patient_number': '354','sign': 'pulse', 'value': '66'},
  {'patient_number': '346','sign': 'blood pressure', 'value': '124/60'},
  {'patient_number': '346','sign': 'pulse', 'value': '72'},
  {'patient_number': '346','sign': 'weight', 'value': '238'},
  {'patient_number': '220','sign': 'blood pressure', 'value': '140/82'},
  {'patient_number': '220','sign': 'temperature', 'value': '98.2'},
  {'patient_number': '378','sign': 'weight', 'value': '213'},
  {'patient_number': '215','sign': 'blood pressure', 'value': '130/80'},
  {'patient_number': '215','sign': 'weight', 'value': '216'},
  {'patient_number': '346','sign': 'temperature', 'value': '98.6'},
  {'patient_number': '346','sign': 'pulse', 'value': '99'},
  {'patient_number': '346','sign': 'blood pressure', 'value': '174/65'},
  {'patient_number': '346','sign': 'respiratory rate', 'value': '20'},
  {'patient_number': '346','sign': 'oxygen saturation', 'value': '91%'},
]

new_vital_signs_df = pd.DataFrame(new_vital_signs)
vitalSign_df = pd.concat([vitalSign_df, new_vital_signs_df], ignore_index=True)

609
23
['Project3_data/192-03.xml', 'Project3_data/148-02.xml', 'Project3_data/346-04.xml', 'Project3_data/189-01.xml', 'Project3_data/236-04.xml', 'Project3_data/338-05.xml', 'Project3_data/139-02.xml', 'Project3_data/385-05.xml', 'Project3_data/376-03.xml', 'Project3_data/397-02.xml', 'Project3_data/278-05.xml', 'Project3_data/242-04.xml', 'Project3_data/110-04.xml', 'Project3_data/244-03.xml', 'Project3_data/354-04.xml', 'Project3_data/346-03.xml', 'Project3_data/220-02.xml', 'Project3_data/324-04.xml', 'Project3_data/139-04.xml', 'Project3_data/234-05.xml', 'Project3_data/378-02.xml', 'Project3_data/215-01.xml', 'Project3_data/346-04.xml']


In [76]:
vitalSign_df.to_csv('vitalSigns.csv', index=False)

In [77]:
sign_frequency = vitalSign_df['sign'].value_counts()
print(sign_frequency)

stable/normal        297
blood pressure       152
pulse                100
temperature           61
respiratory rate      30
oxygen saturation     18
weight                14
Name: sign, dtype: int64
